In [2]:
import Pkg;
Pkg.add("JuMP")
Pkg.add("CPUTime")
Pkg.add("Graphs")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
Precompiling project...
  ✓ CEnum
  ✗ CPLEX
  1 dependency successfully precompiled in 5 seconds. 60 already precompiled.
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package
   Resolving package versions...
   Installed CPUTime ─ v1.0.0
    Updating `~/.julia/environments/v1.8/Project.toml`
  [a9c8d775] + CPUTime v1.0.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [a9c8d775] + CPUTime v1.0.0
Precompiling project...
  ✓ CPUTime
  1 dependency successfully precompiled in 1 seconds. 61 already precompiled. 1 skipped during auto due to previous errors.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
using JuMP
#using CPLEX
using CPUTime
using Graphs

In [4]:
struct Clients
    nb_clients
    X
    Y
    h
    L
    L0
    d
end

In [5]:
function Read_PRP_instance(filename)
    hashmap = Dict()
    open(filename) do f
        
        demands = false
        listing_active = false
        current_value = 0
        
        nbc = 0
        X = Array{Float64}(undef, 0)
        Y = Array{Float64}(undef, 0)
        h = Array{Float64}(undef, 0)
        L = Array{Float64}(undef, 0)
        L0 = Array{Float64}(undef, 0)
        d = Array{Array{Float64}}(undef, 0)
        
        for (i, line) in enumerate(eachline(f))
            x = split(line, " ")
            deleteat!(x, findall(e->e=="", x))
            current_item = x[1]
            
            if(current_item == "0" || listing_active) # Liste de clients / demandes
                listing_active = true
                if(current_item == "d")
                    demands = true
                elseif(demands) # Enregistrement des demandes
                    push!(d, parse.(Float64, x))
                else # enregistrement des clients
                    nbc += 1
                    push!(X, parse(Float64, x[2]))
                    push!(Y, parse(Float64, x[3]))
                    push!(h, parse(Float64, x[6]))
                    push!(L, parse(Float64, x[8]))
                    push!(L0, parse(Float64, x[10]))
                end
            else # variables générales du problème
                try
                    current_value = parse(Int64, x[2])
                catch err
                    current_value = 1^10
                end
                hashmap[current_item] = current_value
            end
        end
        C = Clients(nbc, X, Y, h, L, L0, d)
        hashmap["Clients"] = C
    end 
    return hashmap
end

Read_PRP_instance (generic function with 1 method)

In [6]:
h = Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp")

Dict{Any, Any} with 9 entries:
  "f"       => 3000
  "Q"       => 322
  "C"       => 1
  "k"       => 2085
  "Type"    => 1
  "u"       => 30
  "l"       => 6
  "Clients" => Clients(15, [143.0, 89.0, 76.0, 285.0, 401.0, 16.0, 267.0, 249.0…
  "n"       => 14

In [7]:
h["Clients"].h[14]

6.0

In [8]:
function dist(C, i, j, t, mc)
    if(t == 1)
        return ((C.X[i] - C.X[j])^2 + (C.Y[i] - C.Y[j])^2)^(0.5) + 0.5
    else
        return ((C.X[i] - C.X[j])^2 + (C.Y[i] - C.Y[j])^2)^(0.5) * mc
    end
end

dist (generic function with 1 method)

In [9]:
function calcul_dist(p)
    if(p["Type"] == 1)
        mc = 0
    else
        mc = p["mc"]
    end
    c = Array{Float64}(undef, (p["n"]+1, p["n"]+1))
    for i in 1:p["n"]+1
        for j in 1:p["n"]+1            
            c[i, j] = dist(p["Clients"], i, j, p["Type"], mc)
        end
    end
    return c
end

calcul_dist (generic function with 1 method)

In [10]:
sum(h["Clients"].h[i] for i in 1:1)

3.0

In [22]:
function Resolve_LSP(p)
    LP = Model(CPLEX.Optimizer)
    LP = Model()
    
    c = calcul_dist(p)
    
    n = p["Clients"].nb_clients
    L0 = p["Clients"].L0
    L = p["Clients"].L
    l = p["l"]+1
    M = 10000
    
    @variable(LP, y[1:l], Bin)  # Production démarrée
    @variable(LP, prod[1:l], Bin)  # Quantité produite
    @variable(LP, inv[1:n, 1:l], Int)  # Inventaire des clients
    @variable(LP, x[1:n, 1:n, 1:l], Bin)  # Camion entre i et j au temps t
    @variable(LP, q[1:n, 1:l], Int)

    @objective(LP, Min, sum(p["u"]*prod[t] + p["f"]*y[t] + sum(p["Clients"].h[i]*inv[i, t] for i in 1:n) + sum(sum(c[i, j] * x[i, j, t] for i in 1:n) for j in 1:n) for t in 1:l))
    
    for t in 1:l
        if(t ≠ 1)
            @constraint(LP, inv[1, t-1] + prod[t] == sum(q[i, t] for i in 1:n) + inv[1, t]) # (1) OK
            @constraint(LP,inv[1, t-1] <= L0) #(4)
        end
        for i in 1:n
            if(i ≠ 1 && t ≠ 1)
                @constraint(LP, inv[i, t-1] + q[i, t] == p["Clients"].d[i-1][t] + inv[i, t]) # (2) OK
                @constraint(LP, inv[i, t-1] + q[i, t] <= L[i]) # (5)
            end
            
        end
        @constraint(LP,prod[t] <= M*y[t]) # (3) M est une variable arbitrairement grande
        
        # ...
        
    end
    
    # print(LP)
    println()

    #optimize!(LP)
   
    #println(solution_summary(LP, verbose=true))
    
    return 0
end

Resolve_LSP (generic function with 1 method)

In [20]:
Resolve_LSP(Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp"))

0

In [21]:
Resolve_LSP(Read_PRP_instance("./PRP_instances/B_100_instance17.prp"))

0

In [69]:
Read_PRP_instance("./PRP_instances/B_100_instance17.prp")

Dict{Any, Any} with 10 entries:
  "Clients" => Clients(101, [0.0, 72.0, 58.0, 50.0, 78.0, 36.0, 66.0, 66.0, 16.…
  "f"       => 70000
  "Q"       => 8000
  "C"       => 120000
  "k"       => 9
  "Type"    => 2
  "u"       => 0
  "l"       => 20
  "mc"      => 15
  "n"       => 100